# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [4]:
# Do all imports and installs here
import pandas as pd
import pyspark

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

Scope
This project will integrate I94 immigration data, world temperature data and US demographic data to setup a data warehouse with fact and dimension tables.

Data Sets

I94 Immigration Data
airport-codes_csv
U.S. City Demographic Data
Tools

AWS S3: data storage
AWS Redshift: data warehouse and data analysis
Python for data processing
Pandas - exploratory data analysis on small data set
PySpark - data processing on large data set

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

Data Set	Format	Description

I94 Immigration Data	SAS	Data contains international visitor arrival statistics by world regions and select countries (including top 20), type of visa, mode of transportation, age groups, states visited (first intended address only), and the top ports of entry (for select countries).

airport-codes_csv	CSV	This dataset contains details of all the airport codes, country, city, type , name and coordinates of the airport.

U.S. City Demographic Data	CSV	This dataset contains information about the demographics of all US cities and census-designated places with a population greater or equal to 65,000.


In [ ]:
'''
Step 2: Explore and Assess the Data
 Explore the Data 
1. Use pandas for exploratory data analysis to get an overview on these data sets
2. Split data sets to dimensional tables and change column names for better understanding
3. Utilize PySpark on one of the SAS data sets to test ETL data pipeline logic
'''

In [5]:
# Read in the data here
df_immi = pd.read_csv("immigration_data_sample.csv")

In [6]:
df_immi.head(5)

Unnamed: 0      cicid   i94yr  i94mon  i94cit  i94res i94port  arrdate  \
0     2027561  4084316.0  2016.0     4.0   209.0   209.0     HHW  20566.0   
1     2171295  4422636.0  2016.0     4.0   582.0   582.0     MCA  20567.0   
2      589494  1195600.0  2016.0     4.0   148.0   112.0     OGG  20551.0   
3     2631158  5291768.0  2016.0     4.0   297.0   297.0     LOS  20572.0   
4     3032257   985523.0  2016.0     4.0   111.0   111.0     CHM  20550.0   

   i94mode i94addr    ...     entdepu  matflag  biryear   dtaddto  gender  \
0      1.0      HI    ...         NaN        M   1955.0  07202016       F   
1      1.0      TX    ...         NaN        M   1990.0  10222016       M   
2      1.0      FL    ...         NaN        M   1940.0  07052016       M   
3      1.0      CA    ...         NaN        M   1991.0  10272016       M   
4      3.0      NY    ...         NaN        M   1997.0  07042016       F   

  insnum airline        admnum  fltno  visatype  
0    NaN      JL  5.658267e+10  00782        WT  
1    NaN     *GA  9.436200e+10  XBLNG        B2  
2    NaN      LH  5.578047e+10  00464        WT  
3    NaN      QR  9.478970e+10  00739        B2  
4    NaN     NaN  4.232257e+10   LAND        WT  

[5 rows x 29 columns]

In [7]:
df_immi.columns

Index(['Unnamed: 0', 'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port',
       'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa',
       'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd',
       'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum',
       'airline', 'admnum', 'fltno', 'visatype'],
      dtype='object')

In [8]:
fact_immigration = df_immi[['cicid', 'i94yr', 'i94mon', 'i94port', 'i94addr', 'arrdate', 'depdate', 'i94mode', 'i94visa']]
fact_immigration.columns = ['cic_id', 'year', 'month', 'city_code', 'state_code', 'arrive_date', 'departure_date', 'mode', 'visa']
fact_immigration.head(5)

cic_id    year  month city_code state_code  arrive_date  departure_date  \
0  4084316.0  2016.0    4.0       HHW         HI      20566.0         20573.0   
1  4422636.0  2016.0    4.0       MCA         TX      20567.0         20568.0   
2  1195600.0  2016.0    4.0       OGG         FL      20551.0         20571.0   
3  5291768.0  2016.0    4.0       LOS         CA      20572.0         20581.0   
4   985523.0  2016.0    4.0       CHM         NY      20550.0         20553.0   

   mode  visa  
0   1.0   2.0  
1   1.0   2.0  
2   1.0   2.0  
3   1.0   2.0  
4   3.0   2.0

In [9]:
fact_immigration['country'] = 'United States'
fact_immigration.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


cic_id    year  month city_code state_code  arrive_date  departure_date  \
0  4084316.0  2016.0    4.0       HHW         HI      20566.0         20573.0   
1  4422636.0  2016.0    4.0       MCA         TX      20567.0         20568.0   
2  1195600.0  2016.0    4.0       OGG         FL      20551.0         20571.0   
3  5291768.0  2016.0    4.0       LOS         CA      20572.0         20581.0   
4   985523.0  2016.0    4.0       CHM         NY      20550.0         20553.0   

   mode  visa        country  
0   1.0   2.0  United States  
1   1.0   2.0  United States  
2   1.0   2.0  United States  
3   1.0   2.0  United States  
4   3.0   2.0  United States

In [10]:
dim_immi_personal = df_immi[['cicid', 'i94cit', 'i94res', 'biryear', 'gender', 'insnum']]
dim_immi_personal.columns = [['cic_id', 'citizen_country', 'residence_country', 'birth_year', 'gender', 'ins_num']]
dim_immi_personal.head(5)

cic_id citizen_country residence_country birth_year gender ins_num
0  4084316.0           209.0             209.0     1955.0      F     NaN
1  4422636.0           582.0             582.0     1990.0      M     NaN
2  1195600.0           148.0             112.0     1940.0      M     NaN
3  5291768.0           297.0             297.0     1991.0      M     NaN
4   985523.0           111.0             111.0     1997.0      F     NaN

In [11]:
dim_immi_airline = df_immi[['cicid', 'airline', 'admnum', 'fltno', 'visatype']]
dim_immi_airline.columns = ['cic_id', 'airline', 'admin_num', 'flight_number', 'visa_type']
dim_immi_airline.head(5)

cic_id airline     admin_num flight_number visa_type
0  4084316.0      JL  5.658267e+10         00782        WT
1  4422636.0     *GA  9.436200e+10         XBLNG        B2
2  1195600.0      LH  5.578047e+10         00464        WT
3  5291768.0      QR  9.478970e+10         00739        B2
4   985523.0     NaN  4.232257e+10          LAND        WT

In [12]:
#Exploring Demography Data Set
df_demog = pd.read_csv('us-cities-demographics.csv', delimiter=';')
df_demog.head(5)

City          State  Median Age  Male Population  \
0     Silver Spring       Maryland        33.8          40601.0   
1            Quincy  Massachusetts        41.0          44129.0   
2            Hoover        Alabama        38.5          38040.0   
3  Rancho Cucamonga     California        34.5          88127.0   
4            Newark     New Jersey        34.6         138040.0   

   Female Population  Total Population  Number of Veterans  Foreign-born  \
0            41862.0             82463              1562.0       30908.0   
1            49500.0             93629              4147.0       32935.0   
2            46799.0             84839              4819.0        8229.0   
3            87105.0            175232              5821.0       33878.0   
4           143873.0            281913              5829.0       86253.0   

   Average Household Size State Code                       Race  Count  
0                    2.60         MD         Hispanic or Latino  25924  
1                    2.39         MA                      White  58723  
2                    2.58         AL                      Asian   4759  
3                    3.18         CA  Black or African-American  24437  
4                    2.73         NJ                      White  76402

In [13]:
dim_city_population = df_demog[['City', 'State', 'Male Population', 'Female Population', 'Number of Veterans', 'Foreign-born', 'Race']]
dim_city_population.columns = ['city', 'state', 'male_pop', 'female_pop', 'num_vetarans', 'foreign_born', 'race']
dim_city_population.head(5)

city          state  male_pop  female_pop  num_vetarans  \
0     Silver Spring       Maryland   40601.0     41862.0        1562.0   
1            Quincy  Massachusetts   44129.0     49500.0        4147.0   
2            Hoover        Alabama   38040.0     46799.0        4819.0   
3  Rancho Cucamonga     California   88127.0     87105.0        5821.0   
4            Newark     New Jersey  138040.0    143873.0        5829.0   

   foreign_born                       race  
0       30908.0         Hispanic or Latino  
1       32935.0                      White  
2        8229.0                      Asian  
3       33878.0  Black or African-American  
4       86253.0                      White

In [14]:
dim_city_statistics = df_demog[['City', 'State', 'Median Age', 'Average Household Size']]
dim_city_statistics.columns = ['city', 'state', 'median_age', 'avg_household_size']
dim_city_statistics.head(5)

city          state  median_age  avg_household_size
0     Silver Spring       Maryland        33.8                2.60
1            Quincy  Massachusetts        41.0                2.39
2            Hoover        Alabama        38.5                2.58
3  Rancho Cucamonga     California        34.5                3.18
4            Newark     New Jersey        34.6                2.73

In [15]:
#Exploring Airport codes file
df_airport = pd.read_csv('airport-codes_csv.csv')
df_airport.head(5)

ident           type                                name  elevation_ft  \
0   00A       heliport                   Total Rf Heliport          11.0   
1  00AA  small_airport                Aero B Ranch Airport        3435.0   
2  00AK  small_airport                        Lowell Field         450.0   
3  00AL  small_airport                        Epps Airpark         820.0   
4  00AR         closed  Newport Hospital & Clinic Heliport         237.0   

  continent iso_country iso_region  municipality gps_code iata_code  \
0       NaN          US      US-PA      Bensalem      00A       NaN   
1       NaN          US      US-KS         Leoti     00AA       NaN   
2       NaN          US      US-AK  Anchor Point     00AK       NaN   
3       NaN          US      US-AL       Harvest     00AL       NaN   
4       NaN          US      US-AR       Newport      NaN       NaN   

  local_code                            coordinates  
0        00A     -74.93360137939453, 40.07080078125  
1       00AA                 -101.473911, 38.704022  
2       00AK            -151.695999146, 59.94919968  
3       00AL  -86.77030181884766, 34.86479949951172  
4        NaN                    -91.254898, 35.6087

In [16]:
dim_airport_codes=df_airport[['type','name','iso_country','iso_region','municipality','gps_code','iata_code','local_code','coordinates']]
dim_airport_codes.head(5)

type                                name iso_country iso_region  \
0       heliport                   Total Rf Heliport          US      US-PA   
1  small_airport                Aero B Ranch Airport          US      US-KS   
2  small_airport                        Lowell Field          US      US-AK   
3  small_airport                        Epps Airpark          US      US-AL   
4         closed  Newport Hospital & Clinic Heliport          US      US-AR   

   municipality gps_code iata_code local_code  \
0      Bensalem      00A       NaN        00A   
1         Leoti     00AA       NaN       00AA   
2  Anchor Point     00AK       NaN       00AK   
3       Harvest     00AL       NaN       00AL   
4       Newport      NaN       NaN        NaN   

                             coordinates  
0     -74.93360137939453, 40.07080078125  
1                 -101.473911, 38.704022  
2            -151.695999146, 59.94919968  
3  -86.77030181884766, 34.86479949951172  
4                    -91.254898, 35.6087

In [ ]:
#Read data with Spark

In [17]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [18]:
#write to parquet
#df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")
df_spark.head(1)

[Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')]

#### Cleaning Steps

1. Transform arrive_date, departure_date in immigration data from SAS time format to pandad datetime format
2. Parse I94_SAS_Labels_Descriptions.SAS file to get auxiliary dimension table - country_code, city_code, state_code
3. Tranform city, state in demography data to upper case to match city_code and state_code table

In [19]:
# Performing cleaning tasks here
#1. Transform arrdate, depdate from SAS time format to pandad.datetime

def SAS_to_datetime(date):
    return pd.to_timedelta(date, unit='D') + pd.Timestamp('1960-1-1')

In [20]:
fact_immigration['arrive_date'] = SAS_to_datetime(fact_immigration['arrive_date'])
fact_immigration['departure_date'] = SAS_to_datetime(fact_immigration['departure_date'])
fact_immigration.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


cic_id    year  month city_code state_code arrive_date departure_date  \
0  4084316.0  2016.0    4.0       HHW         HI  2016-04-22     2016-04-29   
1  4422636.0  2016.0    4.0       MCA         TX  2016-04-23     2016-04-24   
2  1195600.0  2016.0    4.0       OGG         FL  2016-04-07     2016-04-27   
3  5291768.0  2016.0    4.0       LOS         CA  2016-04-28     2016-05-07   
4   985523.0  2016.0    4.0       CHM         NY  2016-04-06     2016-04-09   

   mode  visa        country  
0   1.0   2.0  United States  
1   1.0   2.0  United States  
2   1.0   2.0  United States  
3   1.0   2.0  United States  
4   3.0   2.0  United States

In [21]:
#2. Parse description file to get auxiliary dimension table - country_code, city_code, state_code

with open("I94_SAS_Labels_Descriptions.SAS") as f:
    contents = f.readlines()

In [22]:
country_code = {}
for countries in contents[10:298]:
    pair = countries.split('=')
    code, country = pair[0].strip(), pair[1].strip().strip("'")
    country_code[code] = country

In [23]:
df_country_code = pd.DataFrame(list(country_code.items()), columns=['code', 'country'])
df_country_code.head(5)

code      country
0  236  AFGHANISTAN
1  101      ALBANIA
2  316      ALGERIA
3  102      ANDORRA
4  324       ANGOLA

In [24]:
city_code = {}
for cities in contents[303:962]:
    pair = cities.split('=')
    code, city = pair[0].strip("\t").strip().strip("'"), pair[1].strip('\t').strip().strip("''")
    city_code[code] = city

In [25]:
df_city_code = pd.DataFrame(list(city_code.items()), columns=['code', 'city'])
df_city_code.head(5)

code                          city
0  ANC        ANCHORAGE, AK         
1  BAR  BAKER AAF - BAKER ISLAND, AK
2  DAC        DALTONS CACHE, AK     
3  PIZ    DEW STATION PT LAY DEW, AK
4  DTH        DUTCH HARBOR, AK

In [26]:
state_code = {}
for states in contents[982:1036]:
    pair = states.split('=')
    code, state = pair[0].strip('\t').strip("'"), pair[1].strip().strip("'")
    state_code[code] = state

In [27]:
df_state_code = pd.DataFrame(list(state_code.items()), columns=['code', 'state'])
df_state_code.head(5)

code       state
0   AK      ALASKA
1   AZ     ARIZONA
2   AR    ARKANSAS
3   CA  CALIFORNIA
4   CO    COLORADO

In [28]:
#3. Tranform city, state in dimension talbe to upper case to match city_code and state_code table

dim_city_statistics['city'] = dim_city_statistics['city'].str.upper()
dim_city_statistics['state'] = dim_city_statistics['state'].str.upper()
dim_city_statistics.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


city          state  median_age  avg_household_size
0     SILVER SPRING       MARYLAND        33.8                2.60
1            QUINCY  MASSACHUSETTS        41.0                2.39
2            HOOVER        ALABAMA        38.5                2.58
3  RANCHO CUCAMONGA     CALIFORNIA        34.5                3.18
4            NEWARK     NEW JERSEY        34.6                2.73

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

Since the purpose of this data warehouse is for OLAP and BI app usage, we will model these data sets with star schema data modeling.

Star Schema

dim_airport_codes, 
dim_city_statistics,
dim_city_population,
dim_immi_personal
dim_immi_ailine

fact_immigration,


#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Assume all data sets are stored in S3 buckets.
2. Follow by Step 2 – Cleaning step to clean up data sets
3. Transform immigration data to 1 fact table and 2 dimension tables, fact table will be partitioned    by state
4. Parsing label description file to get auxiliary tables
5. Transform temperature data to dimension table
6. Split demography data to 2 dimension tables
7. Store these tables back to target S3 bucket


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

Data processing and data model was created by Spark.

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [29]:
# Perform quality checks here

import os
import configparser
from pathlib import Path
from pyspark.sql import SparkSession

In [31]:
config = configparser.ConfigParser()
config.read('capstone.cfg', encoding='utf-8-sig')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']
SOURCE_S3_BUCKET = config['S3']['SOURCE_S3_BUCKET']
DEST_S3_BUCKET = config['S3']['DEST_S3_BUCKET']

In [32]:
spark = SparkSession.builder\
                    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
                    .enableHiveSupport().getOrCreate()

In [ ]:
# 1. Data schema of every dimensional table matches data model

In [33]:
s3_bucket = Path(SOURCE_S3_BUCKET)

In [36]:
for file_dir in s3_bucket.iterdir():
    if file_dir.is_dir():
        path = str(file_dir)
        df = spark.read.parquet(path)
        print("Table: " + path.split('/')[-1])
        schema = df.printSchema()

Py4JJavaError: An error occurred while calling o162.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:290)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:538)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:611)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:603)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:801)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.appl

In [ ]:
#2 No empty table after running ETL data pipeline

In [35]:
for file_dir in s3_bucket.iterdir():
    if file_dir.is_dir():
        path = str(file_dir)
        df = spark.read.parquet(path)
        record_num = df.count()
        if record_num <= 0:
            raise ValueError("This table is empty!")
        else:
            print("Table: " + path.split('/')[-1] + f" is not empty: total {record_num} records.")

Py4JJavaError: An error occurred while calling o112.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3, localhost, executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:290)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:538)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:611)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:603)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:801)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.appl

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

fact_immigration	type	description
immigration_id	int	primary key
cic_id	bigint	cic id
year	int	4 digit year
month	int	numeric month
city_code	char(3)	3 character abbreviation of city
state_code	char(2)	2 character abbreviation for state
mode	int	traffic method
visa	int	visa category
arrive_date	timestamp	arrive date
departure_date	timestamp	leaving date
country	char(13)	country

dim_immi_personal	type	description
immi_personal_id	int	primary key
cic_id	bigint	cic id
citizen_country	int	country of citizenship
residence_country	int	country of residence
birth_year	int	birth year
gender	char(1)	gender
ins_num	int	INS number

dim_immi_airline	type	Description
immi_airline_id	int	primary key
cic_id	bigint	cic id
airline	varchar	airline used to arrive
admin_num	bigint	admission number
flight_number	varchar	fligh number
visa type	char(12)	type of visa


dim_city_statistics	type	description
city_stat_id	int	primary key
city_code	char(3)	3 character abbreviation of city
state_code	char(2)	2 character abbreviation for state
median_age	int	city median age
avg_household_size	float	average houshold size


dim_city_population	type	description
city_pop_id	int	primary key
city_code	char(3)	3 character abbreviation of city
state_code	char(2)	2 character abbreviation for state
male_population	int	male population
female_population	int	female population


dim_airport_codes	type	description
airport_code_id	int	primary key
type		type of airport
name		name of airport
iso_country		country of airport
iso_region		region of airport
muncipality		muncipality of airport
gps_code		gps code of airport
iata_code		iata code of aiport
local_code		local code of airport


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

In [24]:
'''
Tools and Technologies
1. AWS S3 for data storage
2. Pandas for sample data set exploratory data analysis
3. PySpark for large data set data processing to transform staging table to dimensional table
'''

'\nTools and Technologies\n1. AWS S3 for data storage\n2. Pandas for sample data set exploratory data analysis\n3. PySpark for large data set data processing to transform staging table to dimensional table\n'

In [ ]:
'''
Data Update Frequency
Tables created from immigration  data set should be updated monthly since the raw data set is built up monthly.
Tables created from demography data set and airport codes could be updated annually since demography data collection takes time and high frequent demography might take high cost but generate wrong conclusion.
All tables should be update in an append-only mode.

'''

In [ ]:
'''
Future Design Considerations
The data was increased by 100x. If Spark with standalone server mode can not process 100x data set, we could consider to put data in AWS EMR which is a distributed data cluster for processing large data sets on cloud

The data populates a dashboard that must be updated on a daily basis by 7am every day.

Apache Airflow could be used for building up a ETL data pipeline to regularly update the date and populate a report. Apache Airflow also integrate with Python and AWS very well. More applications can be combined together to deliever more powerful task automation.

The database needed to be accessed by 100+ people.

AWS Redshift can handle up to 500 connections. If this SSOT database will be accessed by 100+ people, we can move this database to Redshift with confidence to handle this request. Cost/Benefit analysis will be needed if we are going be implement this cloud solution.


'''